In [1]:
from datetime import datetime
print(str(datetime.now()))
import numpy as np
# os.environ["TF_CPP_MIN_LOG_LEVEL"] ="3"
import tensorflow as tf
import tensorflow_recommenders as tfrs
import pandas as pd
import math

import os
import glob

2022-08-24 08:34:30.623190


2022-08-24 08:34:31.338245: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-24 08:34:31.338270: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
gpu_available = tf.test.is_gpu_available()
gpu_available
req_cols = ['ITEM_ID', 'USER_ID', 'CABIN_TYPE', 'USER_RESIDENCE']

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2022-08-24 08:34:34.401072: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-24 08:34:34.401100: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-24 08:34:34.401119: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bioss-System-Product-Name): /proc/driver/nvidia/version does not exist


In [3]:
data_set_df_updated = pd.read_csv("dataset/interaction_demo.csv")
test_df = pd.read_csv("dataset/interaction_test_demo.csv")
data_set_df_updated.loc[data_set_df_updated.USER_RESIDENCE.isnull(),"USER_RESIDENCE"] = 'None'
test_df.loc[test_df.USER_RESIDENCE.isnull(),"USER_RESIDENCE"] = 'None'
train_df = pd.concat([data_set_df_updated, test_df], ignore_index=True)
train_df.sort_values("TIMESTAMP", ascending= False, inplace= True)

In [4]:
item_df = train_df[["ITEM_ID"]].drop_duplicates("ITEM_ID")
item_ds = tf.data.Dataset.from_tensor_slices(item_df.to_dict("list")).batch(32)
item_df.shape

(350, 1)

In [5]:
train_ds =  tf.data.Dataset.from_tensor_slices(train_df[req_cols].to_dict("list")).batch(256)

In [6]:
USER_ID_unique = np.unique(np.concatenate(list(train_ds.map(
        lambda x: x["USER_ID"]))))

CABIN_TYPE_unique = np.unique(np.concatenate(list(train_ds.map(
        lambda x: x["CABIN_TYPE"]))))

USER_RESIDENCE_unique = np.unique(np.concatenate(list(train_ds.map(
        lambda x: x["USER_RESIDENCE"]))))

In [7]:
item_unique =  np.unique(np.concatenate(list(train_ds.map(
        lambda x: x["ITEM_ID"]))))

In [8]:
class UserModel(tf.keras.Model):

    def __init__(self):
        super().__init__()

        emb_dim = 32    
        self.user_id_embedding = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(
                vocabulary=USER_ID_unique, mask_token=None),
            tf.keras.layers.Embedding(len(USER_ID_unique) + 1, emb_dim),
        ])
            
        self.cabin_type_embedding = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(
                vocabulary= CABIN_TYPE_unique, mask_token=None),  
            tf.keras.layers.Embedding(len(CABIN_TYPE_unique) + 1, emb_dim),
        ])

        self.user_residence_embedding = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(
                vocabulary=USER_RESIDENCE_unique, mask_token=None),
            tf.keras.layers.Embedding(len(USER_RESIDENCE_unique) + 1, emb_dim),
        ])
        

    def call(self, user_interation_data):
        return tf.concat([                          
            self.user_id_embedding(user_interation_data["USER_ID"]), 
            self.cabin_type_embedding(user_interation_data["CABIN_TYPE"]), 
            self.user_residence_embedding(user_interation_data["USER_RESIDENCE"]),
        ], axis=1)





In [9]:
class ItemModel(tf.keras.Model):

    def __init__(self):
        super().__init__()

        

        self.item_embedding = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(
                vocabulary=item_unique, mask_token=None),
            tf.keras.layers.Embedding(len(item_unique) + 1, 32),
        ])


    def call(self, user_interation_data):

        return tf.concat([
            self.item_embedding(user_interation_data["ITEM_ID"])
            
            ], axis=1)

In [10]:
class TRFSRetrievalModel(tfrs.models.Model):

    def __init__(self, UserModel,ItemModel, item_ds ):
        super().__init__()



        self.query_model = tf.keras.Sequential([#,UserModel()
          UserModel(),
          tf.keras.layers.Dense(32 , kernel_initializer= tf.keras.initializers.RandomNormal(seed=99)),   
        ])
        

        self.candidate_model = tf.keras.Sequential([
          ItemModel(),
          tf.keras.layers.Dense(32, kernel_initializer= tf.keras.initializers.RandomNormal(seed=1))
        ]) 
        
        


#         metrics = [
#           tf.keras.metrics.TopKCategoricalAccuracy(
#               k=x, name=f"factorized_top_k/top_{x}_categorical_accuracy")
#           for x in [3,5,10,15, 25]
#         ]  
        
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
            item_ds.map(self.candidate_model),
                ks= (3, 5, 10,15, 25))
        )
        
#         self.task = tfrs.tasks.Retrieval(
#             metrics=tfrs.metrics.FactorizedTopK(
#                 candidates=item_ds.map(self.candidate_model),
#                 metrics = metrics,
#                 k = 100
#             ),
#             # temperature = 0.5,
#             num_hard_negatives = 5
#         )

    def compute_loss(self, features, training= True):

        item_features = {"ITEM_ID":features.pop("ITEM_ID") }
        query_embeddings = self.query_model(features)
        item_embeddings = self.candidate_model(item_features)

        return self.task(query_embeddings, 
        item_embeddings, 
        compute_metrics=True
        )

    def call(self, test):
        features= test.copy()
        item_features = {"ITEM_ID":features.pop("ITEM_ID") }
        query_embeddings = self.query_model(features)
        item_embeddings = self.candidate_model(item_features)

        return query_embeddings, item_embeddings


In [11]:
test_max_index = math.floor(train_df.shape[0]*0.1)
train_split_len = train_df.shape[0] - test_max_index
data_set_tf = tf.data.Dataset.from_tensor_slices(train_df[req_cols].to_dict("list"))
test = data_set_tf.take(test_max_index)
train = data_set_tf.skip(test_max_index).take(train_split_len)
shuffled = train.shuffle(train_split_len, seed=42, reshuffle_each_iteration=True)
cached_train = shuffled.batch(512).prefetch(4096)#train.shuffle(100_000).batch(2048)
cached_test = test.batch(512).cache()


In [12]:
train_df.shape[0], test_max_index

(34853, 3485)

In [13]:
def delete_all_check_points(fpath= 'new_amazon_check_points/*'):
    files = glob.glob(fpath)
    for f in files:
        os.remove(f)
        

In [14]:
def get_call_back_fun(K):
    delete_all_check_points()
    model_path = f"new_amazon_check_points/best_check_point_{K}k"
    
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=model_path,
        save_weights_only=True,
        monitor=f'val_factorized_top_k/top_{K}_categorical_accuracy',
        mode='max',
        save_best_only=True)

    early_stoping = tf.keras.callbacks.EarlyStopping(monitor=f'val_factorized_top_k/top_{K}_categorical_accuracy',
                                                     mode='min',
                                                     patience=5)

    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor=f'val_factorized_top_k/top_{K}_categorical_accuracy', 
                                                     factor=0.6,
                                                     #mode='min',
                                                     patience=9, 
                                                     min_lr=1e-6
    )
    return model_path, model_checkpoint_callback, early_stoping, reduce_lr

In [15]:
model_path, model_checkpoint_callback, early_stoping, reduce_lr = get_call_back_fun(10)

In [16]:
model = TRFSRetrievalModel(UserModel, ItemModel, item_ds)
model.compile(optimizer=tf.keras.optimizers.Adam(0.01)) 

In [ ]:
model.fit(cached_train,
          validation_data=cached_test,
          epochs=100,
          verbose=1, 
          workers=3,
          use_multiprocessing=True,
          callbacks=[model_checkpoint_callback, 
                     reduce_lr]

          )

Epoch 1/100
62/62 [==============================] - 5s 55ms/step - factorized_top_k/top_3_categorical_accuracy: 0.0593 - factorized_top_k/top_5_categorical_accuracy: 0.1088 - factorized_top_k/top_10_categorical_accuracy: 0.2088 - factorized_top_k/top_15_categorical_accuracy: 0.2777 - factorized_top_k/top_25_categorical_accuracy: 0.3759 - loss: 2944.1422 - regularization_loss: 0.0000e+00 - total_loss: 2944.1422 - val_factorized_top_k/top_3_categorical_accuracy: 0.0970 - val_factorized_top_k/top_5_categorical_accuracy: 0.2020 - val_factorized_top_k/top_10_categorical_accuracy: 0.2755 - val_factorized_top_k/top_15_categorical_accuracy: 0.3461 - val_factorized_top_k/top_25_categorical_accuracy: 0.4393 - val_loss: 2238.2056 - val_regularization_loss: 0.0000e+00 - val_total_loss: 2238.2056 - lr: 0.0100
Epoch 2/100
62/62 [==============================] - 3s 52ms/step - factorized_top_k/top_3_categorical_accuracy: 0.2090 - factorized_top_k/top_5_categorical_accuracy: 0.3003 - factorized_top_

62/62 [==============================] - 3s 49ms/step - factorized_top_k/top_3_categorical_accuracy: 0.7300 - factorized_top_k/top_5_categorical_accuracy: 0.7730 - factorized_top_k/top_10_categorical_accuracy: 0.8276 - factorized_top_k/top_15_categorical_accuracy: 0.8567 - factorized_top_k/top_25_categorical_accuracy: 0.8898 - loss: 1337.1226 - regularization_loss: 0.0000e+00 - total_loss: 1337.1226 - val_factorized_top_k/top_3_categorical_accuracy: 0.1133 - val_factorized_top_k/top_5_categorical_accuracy: 0.1472 - val_factorized_top_k/top_10_categorical_accuracy: 0.2146 - val_factorized_top_k/top_15_categorical_accuracy: 0.2600 - val_factorized_top_k/top_25_categorical_accuracy: 0.3704 - val_loss: 2646.4058 - val_regularization_loss: 0.0000e+00 - val_total_loss: 2646.4058 - lr: 0.0100
Epoch 9/100
62/62 [==============================] - 3s 50ms/step - factorized_top_k/top_3_categorical_accuracy: 0.7403 - factorized_top_k/top_5_categorical_accuracy: 0.7840 - factorized_top_k/top_10_cat

62/62 [==============================] - 3s 50ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8022 - factorized_top_k/top_5_categorical_accuracy: 0.8563 - factorized_top_k/top_10_categorical_accuracy: 0.9079 - factorized_top_k/top_15_categorical_accuracy: 0.9309 - factorized_top_k/top_25_categorical_accuracy: 0.9527 - loss: 1144.9489 - regularization_loss: 0.0000e+00 - total_loss: 1144.9489 - val_factorized_top_k/top_3_categorical_accuracy: 0.1455 - val_factorized_top_k/top_5_categorical_accuracy: 0.1980 - val_factorized_top_k/top_10_categorical_accuracy: 0.2809 - val_factorized_top_k/top_15_categorical_accuracy: 0.3311 - val_factorized_top_k/top_25_categorical_accuracy: 0.4189 - val_loss: 2762.4233 - val_regularization_loss: 0.0000e+00 - val_total_loss: 2762.4233 - lr: 0.0060
Epoch 19/100
62/62 [==============================] - 3s 49ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8073 - factorized_top_k/top_5_categorical_accuracy: 0.8609 - factorized_top_k/top_10_ca

62/62 [==============================] - 3s 49ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8249 - factorized_top_k/top_5_categorical_accuracy: 0.8812 - factorized_top_k/top_10_categorical_accuracy: 0.9295 - factorized_top_k/top_15_categorical_accuracy: 0.9492 - factorized_top_k/top_25_categorical_accuracy: 0.9684 - loss: 1100.4129 - regularization_loss: 0.0000e+00 - total_loss: 1100.4129 - val_factorized_top_k/top_3_categorical_accuracy: 0.1615 - val_factorized_top_k/top_5_categorical_accuracy: 0.2195 - val_factorized_top_k/top_10_categorical_accuracy: 0.2918 - val_factorized_top_k/top_15_categorical_accuracy: 0.3438 - val_factorized_top_k/top_25_categorical_accuracy: 0.4293 - val_loss: 2757.9753 - val_regularization_loss: 0.0000e+00 - val_total_loss: 2757.9753 - lr: 0.0060
Epoch 29/100
22/62 [=========>....................] - ETA: 1s - factorized_top_k/top_3_categorical_accuracy: 0.8414 - factorized_top_k/top_5_categorical_accuracy: 0.8979 - factorized_top_k/top_10_categor

In [96]:
model.load_weights("new_amazon_check_points/best_check_point_10k")

In [97]:
result_summary = model.evaluate(cached_test)

7/7 [==============================] - 0s 41ms/step - factorized_top_k/top_3_categorical_accuracy: 0.2046 - factorized_top_k/top_5_categorical_accuracy: 0.2623 - factorized_top_k/top_10_categorical_accuracy: 0.3432 - factorized_top_k/top_15_categorical_accuracy: 0.3986 - factorized_top_k/top_25_categorical_accuracy: 0.4838 - loss: 3742.8446 - regularization_loss: 0.0000e+00 - total_loss: 3742.8446


In [99]:
{val:result_summary[idx] for idx, val in enumerate([3, 5, 10,15, 25]) }

{3: 0.20459111034870148,
 5: 0.2622668445110321,
 10: 0.34318506717681885,
 15: 0.39856529235839844,
 25: 0.4837876558303833}